In [ ]:
import fitz, os, faiss
import re
import pandas as pd
from transformers import pipeline
from Config import Configs
from Config import ModelLoader as ML
from Libraries import Common_MyUtils as MU, Common_TextProcess as TP
from Libraries import PDF_ExtractData as ExtractData, PDF_MergeData as MergeData, Json_ChunkUnder as ChunkUnder
from Libraries import Faiss_Embedding as F_Embedding, Faiss_Searching as F_Searching
from sentence_transformers import CrossEncoder

In [ ]:
Checkpoint = "vinai/bartpho-syllable"
service = "Categories"
inputs = "VH.pdf"
JsonKey = "paragraphs"
JsonField = "Text"

In [ ]:
config = Configs.ConfigValues(service=service, inputs=inputs)
inputPath = config["inputPath"]
PdfPath = config["PdfPath"]
DocPath = config["DocPath"]
exceptPath = config["exceptPath"]
markerPath = config["markerPath"]
statusPath = config["statusPath"]
RawDataPath = config["RawDataPath"]
RawLvlsPath = config["RawLvlsPath"]
StructsPath = config["StructsPath"]
SegmentPath = config["SegmentPath"]
SchemaPath = config["SchemaPath"]
FaissPath = config["FaissPath"]
MappingPath = config["MappingPath"]
MapDataPath = config["MapDataPath"]
MapChunkPath = config["MapChunkPath"]
MetaPath = config["MetaPath"]
DATA_KEY = config["DATA_KEY"]
EMBE_KEY = config["EMBE_KEY"]
SEARCH_EGINE = config["SEARCH_EGINE"]
RERANK_MODEL = config["RERANK_MODEL"]
RESPON_MODEL = config["RESPON_MODEL"]
EMBEDD_MODEL = config["EMBEDD_MODEL"]
CHUNKS_MODEL = config["CHUNKS_MODEL"]
SUMARY_MODEL = config["SUMARY_MODEL"]
WORD_LIMIT = config["WORD_LIMIT"]

MODEL_DIR = "Models"
MODEL_TYPE = "Sentence_Transformer"
EMBEDD_CACHED_MODEL = f"{MODEL_DIR}/{MODEL_TYPE}/{EMBEDD_MODEL}"
CHUNKS_CACHED_MODEL = F"{MODEL_DIR}/{MODEL_TYPE}/{CHUNKS_MODEL}"
SUMARY_CACHED_MODEL = f"{MODEL_DIR}/{MODEL_TYPE}/{SUMARY_MODEL}"

MAX_INPUT = 1024
MAX_TARGET = 256
MIN_TARGET = 64
TRAIN_EPOCHS = 3
LEARNING_RATE = 3e-5
WEIGHT_DECAY = 0.01
BATCH_SIZE = 4

In [ ]:
def loadHardcodes(file_path, wanted=None):
    data = MU.read_json(file_path)
    if "items" not in data:
        return
    result = {}
    for item in data["items"]:
        key = item["key"]
        if (not wanted) or (key in wanted):
            result[key] = item["values"]
    return result

In [ ]:
exceptData = loadHardcodes(exceptPath, wanted=["common_words", "proper_names", "abbreviations"])
markerData = loadHardcodes(markerPath, wanted=["keywords", "markers"])
statusData = loadHardcodes(statusPath, wanted=["brackets", "sentence_ends"])

In [ ]:
indexer, embeddDevice = ML.init_sentence_model(EMBEDD_MODEL, EMBEDD_CACHED_MODEL)
chunker, chunksDevice = ML.init_sentence_model(CHUNKS_MODEL, CHUNKS_CACHED_MODEL)

In [ ]:
Mapping = MU.read_json(MappingPath)
MapData = MU.read_json(MapDataPath)
MapChunk = MU.read_json(MapChunkPath)
faissIndex = faiss.read_index(FaissPath)

In [ ]:
dataExtractor = ExtractData.B1Extractor(
    exceptData,
    markerData,
    statusData,
    proper_name_min_count=10
)

chunkUnder = ChunkUnder.ChunkUndertheseaBuilder(
    embedder=chunker,
    min_words=512,
    max_words=1024,
    sim_threshold=0.7
)

reranker = CrossEncoder(RERANK_MODEL, device=str(embeddDevice))
engine = F_Searching.SemanticSearchEngine(
    indexer=indexer,
    reranker=reranker,
    device=str(embeddDevice),
    normalize=True,
    top_k=20,
    rerank_k=10,
    rerank_batch_size=16
)

In [ ]:
def extractRun(pdf_doc):
    extractedData = dataExtractor.extract(pdf_doc)
    RawDataDict = MergeData.mergeLinesToParagraphs(extractedData)
    return RawDataDict

In [ ]:
def modelTest(sample_text=None, ModelPath=None, max_length=256, min_length=64):
    summarizer_pipeline = pipeline("summarization", model=ModelPath)
    summary = summarizer_pipeline(
        sample_text,
        max_length=max_length,
        min_length=min_length,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    return summary[0]['summary_text']

In [ ]:
def summarize_recursive(text, depth=0, max_depth=5):
    word_count = len(text.split())
    indent = "  " * depth
    print(f"{indent}🔹 Level {depth}: {word_count} từ")

    if word_count < 512:
        return text
    elif word_count < 1024:
        return modelTest(text, SUMARY_CACHED_MODEL, MAX_TARGET, MIN_TARGET)
    else:
        chunks = chunkUnder.build(text)
        summaries = []

        for item in chunks:
            content = item["Content"]
            idx = item.get("Index", "?")
            print(f"{indent}  🔸 Chunk {idx}: {len(content.split())} từ")
            try:
                sub_summary = modelTest(content, SUMARY_CACHED_MODEL, MAX_TARGET, MIN_TARGET)
            except Exception as e:
                return "Bruh"
            
            summaries.append(sub_summary)

        merged_summary = "\n".join(summaries)
        merged_len = len(merged_summary.split())
        if merged_len > 1024 and depth < max_depth:
            return summarize_recursive(merged_summary, depth + 1, max_depth)
        
        return merged_summary

In [ ]:
def runSearch(query):
    results = engine.search(
        query=query,
        faissIndex=faissIndex,
        Mapping=Mapping,
        MapData=MapData,
        top_k=20
    )
    return results

In [ ]:
def runRerank(query, results):
    reranked = engine.rerank(
        query=query,
        results=results,
        top_k=10
    )
    return reranked

In [ ]:
pdf_doc = fitz.open(inputPath)
RawDataDict = extractRun(pdf_doc)
pdf_doc.close()
full_text = TP.merge_txt(RawDataDict, JsonKey, JsonField)

In [ ]:
if len(full_text.split()) > 512:
    final_summary = summarize_recursive(text=full_text)
else:
    final_summary = modelTest(full_text, SUMARY_CACHED_MODEL, MAX_TARGET, MIN_TARGET)
print("\n✨ FINAL SUMMARY ✨\n")
print(final_summary)

In [ ]:
resuls = runSearch(final_summary)
reranked = runRerank(final_summary, resuls)

best_text = reranked[0]["text"] if reranked else ""
print(best_text)